In [52]:
import os
import pandas as pd
import numpy as np
%pylab inline
import statsmodels.api as sm
import statsmodels.formula.api as smf

Populating the interactive namespace from numpy and matplotlib


In [53]:
complaints = pd.read_csv('http://cosmo.nyu.edu/~fb55/PUI2016/data/311_Service_Requests_from_2010_to_Present.csv')

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (17,40,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [54]:
complaints.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
0,34212771,09/01/2016 12:00:06 AM,09/01/2016 06:02:10 AM,NYPD,New York City Police Department,Noise - Residential,Loud Television,Residential Building/House,11413.0,137-47 CARSON STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.675208,-73.754949,"(40.67520813178531, -73.75494944502233)"
1,34212392,09/01/2016 12:00:35 AM,09/02/2016 04:53:48 PM,NYPD,New York City Police Department,Blocked Driveway,No Access,Street/Sidewalk,10469.0,943 EAST 217 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.879986,-73.856707,"(40.87998640855491, -73.85670741555177)"
2,34214520,09/01/2016 12:00:37 AM,09/01/2016 01:05:43 AM,NYPD,New York City Police Department,Blocked Driveway,No Access,Street/Sidewalk,11369.0,98-02 25 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.764632,-73.871623,"(40.76463242381882, -73.87162284918391)"
3,34212483,09/01/2016 12:00:54 AM,09/01/2016 03:15:32 AM,NYPD,New York City Police Department,Blocked Driveway,No Access,Street/Sidewalk,10466.0,1017 EAST 226 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.885559,-73.850782,"(40.885558858976104, -73.85078238918491)"
4,34214231,09/01/2016 12:01:15 AM,09/06/2016 09:27:00 AM,DOT,Department of Transportation,Street Condition,Pothole,NaN,10040.0,4700 BROADWAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.864140,-73.929501,"(40.864140093130686, -73.92950060219349)"


In [55]:
complaints.columns

Index(['Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Agency Name',
       'Complaint Type', 'Descriptor', 'Location Type', 'Incident Zip',
       'Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2',
       'Intersection Street 1', 'Intersection Street 2', 'Address Type',
       'City', 'Landmark', 'Facility Type', 'Status', 'Due Date',
       'Resolution Description', 'Resolution Action Updated Date',
       'Community Board', 'Borough', 'X Coordinate (State Plane)',
       'Y Coordinate (State Plane)', 'Park Facility Name', 'Park Borough',
       'School Name', 'School Number', 'School Region', 'School Code',
       'School Phone Number', 'School Address', 'School City', 'School State',
       'School Zip', 'School Not Found', 'School or Citywide Complaint',
       'Vehicle Type', 'Taxi Company Borough', 'Taxi Pick Up Location',
       'Bridge Highway Name', 'Bridge Highway Direction', 'Road Ramp',
       'Bridge Highway Segment', 'Garage Lot Name', 'Ferry 

In [56]:
# Now we make sure we have unique values for all community districts and boroughs.
solecd = complaints['Community Board'].drop_duplicates().as_matrix()
solecd

array(['13 QUEENS', '12 BRONX', '03 QUEENS', '12 MANHATTAN',
       '09 MANHATTAN', '08 QUEENS', '01 QUEENS', '07 BRONX', '01 BROOKLYN',
       '03 MANHATTAN', '09 QUEENS', '05 BRONX', '03 BROOKLYN',
       '07 MANHATTAN', '12 BROOKLYN', '0 Unspecified', '09 BROOKLYN',
       '07 QUEENS', '15 BROOKLYN', '08 MANHATTAN', '10 BROOKLYN',
       '08 BROOKLYN', '04 MANHATTAN', '02 MANHATTAN', '12 QUEENS',
       '04 BRONX', '04 BROOKLYN', '01 BRONX', 'Unspecified MANHATTAN',
       '10 QUEENS', '18 BROOKLYN', '11 BROOKLYN', '10 MANHATTAN',
       '11 QUEENS', '16 BROOKLYN', '05 BROOKLYN', '05 MANHATTAN',
       '55 BROOKLYN', '08 BRONX', '09 BRONX', '17 BROOKLYN',
       '01 MANHATTAN', '05 QUEENS', '06 QUEENS', '02 BRONX', '11 BRONX',
       '04 QUEENS', 'Unspecified BROOKLYN', '02 QUEENS',
       '01 STATEN ISLAND', '03 STATEN ISLAND', '14 BROOKLYN',
       '07 BROOKLYN', '02 STATEN ISLAND', '10 BRONX', '11 MANHATTAN',
       '06 BROOKLYN', '13 BROOKLYN', '06 MANHATTAN', 'Unspecified BRONX

In [57]:
print(len(solecd))

74


## How many unique values do we have? 
We have 74 community districts in our data 
## Why do we have so many? Some of them are unspecified, missing. Some might be invalid entries. (We should have 59 Community Districts.)
We have 74 community districts instead of 59 because there are unspecified values in the data. 
## Check for duplicates? Are these plausible?
We do not have duplicates in the data after dropping them solecd dataframe.


In [58]:
print(len(complaints.drop_duplicates()))

34897


In [59]:
complaints.isnull().sum()

Unique Key                            0
Created Date                          0
Closed Date                       12779
Agency                                0
Agency Name                           0
Complaint Type                        0
Descriptor                         1171
Location Type                      6801
Incident Zip                       1852
Incident Address                   7174
Street Name                        7175
Cross Street 1                    14533
Cross Street 2                    14638
Intersection Street 1             29857
Intersection Street 2             29885
Address Type                       1556
City                               1845
Landmark                          34881
Facility Type                     19136
Status                                0
Due Date                          13719
Resolution Description             1807
Resolution Action Updated Date     5745
Community Board                       0
Borough                               0


## Is it possible to replace missing values? Is it possible to use Complaint Type 

## to fill missings in Resolution Descriptor?

In [60]:
complaints = complaints.rename(columns={'Complaint Type': 'Complaint_Type', 'Resolution Description': 'Resolution_Description'})

In [61]:
complaints.Resolution_Description.fillna(complaints.Complaint_Type, inplace=True)

In [62]:
complaints.Resolution_Description.isnull().sum()

0

### Generate marker for unplausible Community Districts
### How do these districts look like? 

For this portion of the lab we had to identify the community districts containing the word "unspecified". Following Kisti Korsberg's instructions, we dropped these rows using pandas' series.str.contains() 

In [63]:
complaints = complaints.rename(columns={'Community Board': 'Community_Board'})

In [64]:
complaints = complaints[complaints["Community_Board"].str.contains('Unspecified') != True]

In [65]:

print(len(complaints))

32899


In [66]:
complaints['Community_Board'].drop_duplicates().as_matrix()

array(['13 QUEENS', '12 BRONX', '03 QUEENS', '12 MANHATTAN',
       '09 MANHATTAN', '08 QUEENS', '01 QUEENS', '07 BRONX', '01 BROOKLYN',
       '03 MANHATTAN', '09 QUEENS', '05 BRONX', '03 BROOKLYN',
       '07 MANHATTAN', '12 BROOKLYN', '09 BROOKLYN', '07 QUEENS',
       '15 BROOKLYN', '08 MANHATTAN', '10 BROOKLYN', '08 BROOKLYN',
       '04 MANHATTAN', '02 MANHATTAN', '12 QUEENS', '04 BRONX',
       '04 BROOKLYN', '01 BRONX', '10 QUEENS', '18 BROOKLYN',
       '11 BROOKLYN', '10 MANHATTAN', '11 QUEENS', '16 BROOKLYN',
       '05 BROOKLYN', '05 MANHATTAN', '55 BROOKLYN', '08 BRONX',
       '09 BRONX', '17 BROOKLYN', '01 MANHATTAN', '05 QUEENS', '06 QUEENS',
       '02 BRONX', '11 BRONX', '04 QUEENS', '02 QUEENS',
       '01 STATEN ISLAND', '03 STATEN ISLAND', '14 BROOKLYN',
       '07 BROOKLYN', '02 STATEN ISLAND', '10 BRONX', '11 MANHATTAN',
       '06 BROOKLYN', '13 BROOKLYN', '06 MANHATTAN', '02 BROOKLYN',
       '06 BRONX', '14 QUEENS', '03 BRONX', '83 QUEENS', '80 QUEENS',
      

In [67]:
complaints = complaints[complaints["Community_Board"].str.contains('55') != True]
complaints = complaints[complaints["Community_Board"].str.contains('83') != True]
complaints = complaints[complaints["Community_Board"].str.contains('80') != True]
complaints = complaints[complaints["Community_Board"].str.contains('82') != True]
complaints = complaints[complaints["Community_Board"].str.contains('81') != True]
complaints = complaints[complaints["Community_Board"].str.contains('95') != True]
complaints = complaints[complaints["Community_Board"].str.contains('64') != True]
complaints = complaints[complaints["Community_Board"].str.contains('26') != True]
complaints = complaints[complaints["Community_Board"].str.contains('27') != True]

In [68]:
complaints['Community_Board'].drop_duplicates().as_matrix()

array(['13 QUEENS', '12 BRONX', '03 QUEENS', '12 MANHATTAN',
       '09 MANHATTAN', '08 QUEENS', '01 QUEENS', '07 BRONX', '01 BROOKLYN',
       '03 MANHATTAN', '09 QUEENS', '05 BRONX', '03 BROOKLYN',
       '07 MANHATTAN', '12 BROOKLYN', '09 BROOKLYN', '07 QUEENS',
       '15 BROOKLYN', '08 MANHATTAN', '10 BROOKLYN', '08 BROOKLYN',
       '04 MANHATTAN', '02 MANHATTAN', '12 QUEENS', '04 BRONX',
       '04 BROOKLYN', '01 BRONX', '10 QUEENS', '18 BROOKLYN',
       '11 BROOKLYN', '10 MANHATTAN', '11 QUEENS', '16 BROOKLYN',
       '05 BROOKLYN', '05 MANHATTAN', '08 BRONX', '09 BRONX',
       '17 BROOKLYN', '01 MANHATTAN', '05 QUEENS', '06 QUEENS', '02 BRONX',
       '11 BRONX', '04 QUEENS', '02 QUEENS', '01 STATEN ISLAND',
       '03 STATEN ISLAND', '14 BROOKLYN', '07 BROOKLYN',
       '02 STATEN ISLAND', '10 BRONX', '11 MANHATTAN', '06 BROOKLYN',
       '13 BROOKLYN', '06 MANHATTAN', '02 BROOKLYN', '06 BRONX',
       '14 QUEENS', '03 BRONX'], dtype=object)

In [69]:
print(len(complaints['Community_Board'].drop_duplicates().as_matrix()))

59


### Produce your result: Generate an indicator which ranks the Community District by complaint numbers 
### on the Community district level (cdcomp)

In [70]:
cdcomp = pd.DataFrame(complaints['Unique Key'].groupby(complaints['Community_Board']).count()).reset_index()

In [71]:
cdcomp = cdcomp.rename(columns={'Community_Board': 'Community Board Name', 'Unique Key': 'Count of Complaints'})

In [72]:
cdcomp = cdcomp.sort_values(by='Count of Complaints', ascending=0)

### The chart below displays the number of complaints by community district

In [73]:
cdcomp.head()

,Community Board Name,Count of Complaints
49,12 MANHATTAN,1162
50,12 QUEENS,968
57,17 BROOKLYN,883
20,05 BROOKLYN,871
11,03 BROOKLYN,865


### Think about the demographics of the Community Districts

In [74]:
demographics = pd.read_csv("http://cosmo.nyu.edu/~fb55/PUI2016/data/Final_Demographics.csv")

In [75]:
demographics.head()

,FIPS,cd_id,Total Population,Population Density (per sq. mile),% Total Population: Male,% Total Population: 18 to 24 Years,% Total Population: 25 to 34 Years,% Total Population: 35 to 44 Years,% Population 5 Years And Over: Speak Only English,% Population 5 Years And Over: Spanish or Spanish Creole,...,"Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations","% Employed Civilian Population 16 Years And Over: Management, professional, and related occupations",% Employed Civilian Population 16 Years And Over: Service occupations,% Employed Civilian Population 16 Years And Over: Sales and office occupations,"% Employed Civilian Population 16 Years And Over: Farming, fishing, and forestry occupations","% Employed Civilian Population 16 Years And Over: Construction, extraction, and maintenance occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations"
0,3603701,BX08,106737,31229.95006,46.65,10.73,15.04,11.32,46.80,39.24,...,665,1518,42.66,28.95,20.89,0.24,2.65,4.60,1.40,3.20
1,3603702,BX12,134644,19966.67839,46.35,11.35,14.29,12.57,73.09,18.19,...,1156,4174,29.57,33.98,20.40,0.00,7.08,8.97,1.95,7.02
2,3603703,BX10,121209,12913.81703,45.20,8.62,13.74,12.78,61.79,26.43,...,941,3433,36.20,22.85,25.09,0.00,7.68,8.18,1.76,6.42
3,3603704,BX11,135839,35677.95453,50.09,8.10,17.43,14.09,43.22,36.45,...,2189,5592,30.06,27.86,22.24,0.00,7.03,12.81,3.60,9.20
4,3603705,BX03,172247,39405.79222,44.72,14.24,14.89,12.38,36.82,54.24,...,1437,5436,16.80,41.00,22.29,0.03,8.45,11.43,2.39,9.04


In [76]:
demographics.columns

Index(['FIPS', 'cd_id', 'Total Population',
       'Population Density (per sq. mile)', '% Total Population: Male',
       '% Total Population: 18 to 24 Years',
       '% Total Population: 25 to 34 Years',
       '% Total Population: 35 to 44 Years',
       '% Population 5 Years And Over: Speak Only English',
       '% Population 5 Years And Over: Spanish or Spanish Creole',
       ...
       'Employed Civilian Population 16 Years And Over: Production, transportation, and material moving  occupations: Production occupations',
       'Employed Civilian Population 16 Years And Over: Production, transportation, and material moving  occupations: Transportation and material moving occupations',
       '% Employed Civilian Population 16 Years And Over: Management, professional, and related occupations',
       '% Employed Civilian Population 16 Years And Over: Service occupations',
       '% Employed Civilian Population 16 Years And Over: Sales and office occupations',
       '% Employed Civ

### How many community districts are in file?

In [77]:
cd_id = demographics['cd_id'].drop_duplicates().as_matrix()

In [78]:
cd_id

array(['BX08', 'BX12', 'BX10', 'BX11', 'BX03', 'BX06', 'BX07', 'BX05',
       'BX04', 'BX09', 'BX01', 'BX02', 'MN12', 'MN09', 'MN10', 'MN11111',
       'MN08', 'MN07', 'MN05', 'MN04', 'MN06', 'MN03', 'MN01', 'MN02',
       'SI03', 'SI02', 'SI01', 'BK01', 'BK04', 'BK03', 'BK02', 'BK06',
       'BK08', 'BK16', 'BK05', 'BK18', 'BK17', 'BK09', 'BK07', 'BK10',
       'BK12', 'BK14', 'BK15', 'BK11', 'BK13', 'QN01', 'QN03', 'QN07',
       'QN11', 'QN13', 'QN08', 'QN04', 'QN06', 'QN02', 'QN05', 'QN09',
       'QN12', 'QN10', 'QN14'], dtype=object)

In [79]:
#Replacing community district codes with more than two digits.
demographics = demographics.replace(['MN11111'], ['MN11'])

In [80]:
print(len(cd_id))

59


There are 59 community districts in the demographics data. 

### Manipulate data to get some information on demographics by Community District. 
### Think about who might be more likely to call 311 and  Save data frame

In [81]:
demographics = demographics.iloc[:,[1, 139]]
demographics = demographics.rename(columns={'Median household income (In 2014 Inflation Adjusted Dollars)': 'Median Household Income', 'cd_id': 'Community Board Name'})
demographics.head()

,Community Board Name,Median Household Income
0,BX08,54224
1,BX12,44906
2,BX10,54962
3,BX11,47910
4,BX03,22041


In [82]:
broadband = pd.read_csv("http://cosmo.nyu.edu/~fb55/PUI2016/data/ACS_Computer_Use_and_Internet_2014_1Year_Estimate.csv")

In [83]:
print(len(broadband))

59


In [84]:
broadband.isnull().sum()

FIPS                                                       0
Geographic Identifier                                      0
Qualifying Name                                            0
Households                                                 0
Households: With An Internet Subscription                  0
Households: Dial-Up Alone                                  0
Households: Dsl                                            0
Households: With Mobile Broadband                          0
Households: Without Mobile Broadband                       0
Households: Cable Modem                                    0
Households: With Mobile Broadband.1                        0
Households: Without Mobile Broadband.1                     0
Households: Fiber-Optic                                    0
Households: With Mobile Broadband.2                        0
Households: Without Mobile Broadband.2                     0
Households: Satellite Internet Service                     0
Households: With Mobile 

In [85]:
broadband.columns

Index(['FIPS', 'Geographic Identifier', 'Qualifying Name', 'Households',
       'Households: With An Internet Subscription',
       'Households: Dial-Up Alone', 'Households: Dsl',
       'Households: With Mobile Broadband',
       'Households: Without Mobile Broadband', 'Households: Cable Modem',
       'Households: With Mobile Broadband.1',
       'Households: Without Mobile Broadband.1', 'Households: Fiber-Optic',
       'Households: With Mobile Broadband.2',
       'Households: Without Mobile Broadband.2',
       'Households: Satellite Internet Service',
       'Households: With Mobile Broadband.3',
       'Households: Without Mobile Broadband.3',
       'Households: Two or More Fixed Broadband Types, or Other',
       'Households: With Mobile Broadband.4',
       'Households: Without Mobile Broadband.4',
       'Households: Mobile Broadband Alone or With Dialup',
       'Households: Internet Access Without A Subscription',
       'Households: No Internet Access',
       '% Househol

In [86]:
broadband = broadband.replace(['NYC-Bronx Community District 8--Riverdale, New York',
       'NYC-Bronx Community District 12--Wakefield, New York',
       'NYC-Bronx Community District 10--Co-op City, New York',
       'NYC-Bronx Community District 11--Pelham Parkway, New York',
       'NYC-Bronx Community District 3 & 6--Belmont, New York',
       'NYC-Bronx Community District 7--Bedford Park, New York',
       'NYC-Bronx Community District 5--Morris Heights, New York',
       'NYC-Bronx Community District 4--Concourse, New York',
       'NYC-Bronx Community District 9--Castle Hill, New York',
       'NYC-Bronx Community District 1 & 2--Hunts Point, New York',
       'NYC-Manhattan Community District 12--Washington Heights, New York',
       'NYC-Manhattan Community District 9--Hamilton Heights, New York',
       'NYC-Manhattan Community District 10--Central Harlem PUMA, New York',
       'NYC-Manhattan Community District 11--East Harlem PUMA, New York',
       'NYC-Manhattan Community District 8--Upper East Side PUMA, New York',
       'NYC-Manhattan Community District 7--Upper West Side & West Side PUMA, New York',
       'NYC-Manhattan Community District 4 & 5--Chelsea, New York',
       'NYC-Manhattan Community District 6--Murray Hill, New York',
       'NYC-Manhattan Community District 3--Chinatown & Lower East Side PUMA, New York',
       'NYC-Manhattan Community District 1 & 2--Battery Park City, New York',
       'NYC-Staten Island Community District 3--Tottenville, New York',
       'NYC-Staten Island Community District 2--New Springville & South Beach PUMA, New York',
       'NYC-Staten Island Community District 1--Port Richmond, New York',
       'NYC-Brooklyn Community District 1--Greenpoint & Williamsburg PUMA, New York',
       'NYC-Brooklyn Community District 4--Bushwick PUMA, New York',
       'NYC-Brooklyn Community District 3--Bedford-Stuyvesant PUMA, New York',
       'NYC-Brooklyn Community District 2--Brooklyn Heights & Fort Greene PUMA, New York',
       'NYC-Brooklyn Community District 6--Park Slope, New York',
       'NYC-Brooklyn Community District 8--Crown Heights North & Prospect Heights PUMA, New York',
       'NYC-Brooklyn Community District 16--Brownsville & Ocean Hill PUMA, New York',
       'NYC-Brooklyn Community District 5--East New York & Starrett City PUMA, New York',
       'NYC-Brooklyn Community District 18--Canarsie & Flatlands PUMA, New York',
       'NYC-Brooklyn Community District 17--East Flatbush, New York',
       'NYC-Brooklyn Community District 9--Crown Heights South, New York',
       'NYC-Brooklyn Community District 7--Sunset Park & Windsor Terrace PUMA, New York',
       'NYC-Brooklyn Community District 10--Bay Ridge & Dyker Heights PUMA, New York',
       'NYC-Brooklyn Community District 12--Borough Park, New York',
       'NYC-Brooklyn Community District 14--Flatbush & Midwood PUMA, New York',
       'NYC-Brooklyn Community District 15--Sheepshead Bay, New York',
       'NYC-Brooklyn Community District 11--Bensonhurst & Bath Beach PUMA, New York',
       'NYC-Brooklyn Community District 13--Brighton Beach & Coney Island PUMA, New York',
       'NYC-Queens Community District 1--Astoria & Long Island City PUMA, New York',
       'NYC-Queens Community District 3--Jackson Heights & North Corona PUMA, New York',
       'NYC-Queens Community District 7--Flushing, New York',
       'NYC-Queens Community District 11--Bayside, New York',
       'NYC-Queens Community District 13--Queens Village, New York',
       'NYC-Queens Community District 8--Briarwood, New York',
       'NYC-Queens Community District 4--Elmhurst & South Corona PUMA, New York',
       'NYC-Queens Community District 6--Forest Hills & Rego Park PUMA, New York',
       'NYC-Queens Community District 2--Sunnyside & Woodside PUMA, New York',
       'NYC-Queens Community District 5--Ridgewood, New York',
       'NYC-Queens Community District 9--Richmond Hill & Woodhaven PUMA, New York',
       'NYC-Queens Community District 12--Jamaica, New York',
       'NYC-Queens Community District 10--Howard Beach & Ozone Park PUMA, New York',
       'NYC-Queens Community District 14--Far Rockaway, New York'], ['BX08', 'BX12', 'BX10', 'BX11', 'BX06', 'BX07', 'BX05', 'BX04', 'BX09', 'BX02',
'MN12', 'MN09', 'MN10', 'MN11', 'MN08', 'MN07', 'MN04', 'MN06', 'MN03', 'MN01',
'SI03', 'SI02', 'SI01', 'BK01', 'BK04', 'BK03', 'BK02', 'BK06', 'BK08', 'BK16', 'BK05', 'BK18',
'BK17', 'BK09', 'BK07', 'BK10', 'BK12', 'BK14', 'BK15', 'BK11', 'BK13', 'QN01', 'QN03', 'QN07',
'QN11', 'QN13', 'QN08', 'QN04', 'QN06', 'QN02', 'QN05', 'QN09', 'QN12', 'QN10', 'QN14'])

In [87]:
broadband = broadband.rename(columns={'Qualifying Name': 'Community Board Name'})

In [88]:
broadband.head()

,FIPS,Geographic Identifier,Community Board Name,Households,Households: With An Internet Subscription,Households: Dial-Up Alone,Households: Dsl,Households: With Mobile Broadband,Households: Without Mobile Broadband,Households: Cable Modem,...,Households: Mobile Broadband Alone or With Dialup,Households: Internet Access Without A Subscription,Households: No Internet Access,% Households: With An Internet Subscription,Households.1,Households: Has A Computer,Households: With Dial-Up Internet Subscription Alone,Households: With A Broadband Internet Subscription,Households: Without An Internet Subscription,Households: No Computer
0,3603701,79500US3603701,BX08,42035,31795,444,2813,946,1867,19178,...,2168,2119,8121,75.64,42035,35048,404,30943,3701,6987
1,3603702,79500US3603702,BX12,44830,32243,178,849,405,444,18653,...,928,1891,10696,71.92,44830,36700,178,31435,5087,8130
2,3603703,79500US3603703,BX10,47050,32729,158,1863,398,1465,20044,...,639,2882,11439,69.56,47050,38700,158,32333,6209,8350
3,3603704,79500US3603704,BX11,44922,32003,141,1478,474,1004,17917,...,1001,2722,10197,71.24,44922,37237,122,31278,5837,7685
4,3603705,79500US3603705,BX06,57556,35503,88,1297,651,646,15847,...,1385,3312,18741,61.68,57556,42576,88,33408,9080,14980


### How many community districts are in file? 

In [89]:
broadband['Community Board Name'].drop_duplicates().as_matrix()
print(len(broadband['Community Board Name'].drop_duplicates().as_matrix()))

55


There are 55 community districts in the broadband access data

### Manipulate data to get some information on demographics by Community District. 
### Think about who might be more likely to call 311
### Aggregate the mobile subscription data

In [98]:
broadband.columns

Index(['FIPS', 'Geographic Identifier', 'Community Board Name', 'Households',
       'Households: With An Internet Subscription',
       'Households: Dial-Up Alone', 'Households: Dsl',
       'Households: With Mobile Broadband',
       'Households: Without Mobile Broadband', 'Households: Cable Modem',
       'Households: With Mobile Broadband.1',
       'Households: Without Mobile Broadband.1', 'Households: Fiber-Optic',
       'Households: With Mobile Broadband.2',
       'Households: Without Mobile Broadband.2',
       'Households: Satellite Internet Service',
       'Households: With Mobile Broadband.3',
       'Households: Without Mobile Broadband.3',
       'Households: Two or More Fixed Broadband Types, or Other',
       'Households: With Mobile Broadband.4',
       'Households: Without Mobile Broadband.4',
       'Households: Mobile Broadband Alone or With Dialup',
       'Households: Internet Access Without A Subscription',
       'Households: No Internet Access',
       '% Hou

In [99]:
with_mobile = broadband.iloc[:,[2, 7, 10, 13, 16, 19, 21]]

In [100]:
with_mobile.columns

Index(['Community Board Name', 'Households: With Mobile Broadband',
       'Households: With Mobile Broadband.1',
       'Households: With Mobile Broadband.2',
       'Households: With Mobile Broadband.3',
       'Households: With Mobile Broadband.4',
       'Households: Mobile Broadband Alone or With Dialup'],
      dtype='object')

In [101]:
with_mobile['Households With Mobile Broadband'] = with_mobile.sum(axis=1)

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [102]:
with_mobile.head()


,Community Board Name,Households: With Mobile Broadband,Households: With Mobile Broadband.1,Households: With Mobile Broadband.2,Households: With Mobile Broadband.3,Households: With Mobile Broadband.4,Households: Mobile Broadband Alone or With Dialup,Households With Mobile Broadband
0,BX08,946,10433,433,37,3510,2168,17527
1,BX12,405,5577,2358,0,2146,928,11414
2,BX10,398,6377,1200,0,3450,639,12064
3,BX11,474,5624,2272,241,2137,1001,11749
4,BX06,651,6690,695,111,6760,1385,16292


In [103]:
print(len(with_mobile))

59


In [104]:
without_mobile =  broadband.iloc[:,[2, 8, 11, 14, 17, 20, 22]]

In [105]:
without_mobile['Households Without Mobile Broadband'] = without_mobile.sum(axis=1)

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [106]:
without_mobile.head()

,Community Board Name,Households: Without Mobile Broadband,Households: Without Mobile Broadband.1,Households: Without Mobile Broadband.2,Households: Without Mobile Broadband.3,Households: Without Mobile Broadband.4,Households: Internet Access Without A Subscription,Households Without Mobile Broadband
0,BX08,1867,8745,506,38,2668,2119,15943
1,BX12,444,13076,3858,131,3142,1891,22542
2,BX10,1465,13667,2290,0,3085,2882,23389
3,BX11,1004,12293,3163,67,3586,2722,22835
4,BX06,646,9157,815,83,8422,3312,22435


In [107]:
print(len(without_mobile))

59


In [108]:
mobile = pd.concat([with_mobile, without_mobile], axis=1)

In [109]:
mobile = mobile.iloc[:,[0, 7, 15]]

In [110]:
mobile.head()

,Community Board Name,Households With Mobile Broadband,Households Without Mobile Broadband
0,BX08,17527,15943
1,BX12,11414,22542
2,BX10,12064,23389
3,BX11,11749,22835
4,BX06,16292,22435


In [111]:
print(len(mobile))

59


### Aggregate internet type by high and low connections


In [119]:
high =  broadband.iloc[:,[2, 6, 9, 12]]
high['Households With High Speed Internet'] = high.sum(axis=1)

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [120]:
high.head()

,Community Board Name,Households: Dsl,Households: Cable Modem,Households: Fiber-Optic,Households With High Speed Internet
0,BX08,2813,19178,939,22930
1,BX12,849,18653,6216,25718
2,BX10,1863,20044,3490,25397
3,BX11,1478,17917,5435,24830
4,BX06,1297,15847,1510,18654


In [114]:
low = broadband.iloc[:, [2, 5, 15]]
low['Households With Low Speed Internet'] = low.sum(axis=1)

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [115]:
low.head()

,Community Board Name,Households: Dial-Up Alone,Households: Satellite Internet Service,Households With Low Speed Internet
0,BX08,444,75,519
1,BX12,178,131,309
2,BX10,158,0,158
3,BX11,141,308,449
4,BX06,88,194,282


In [121]:
conection = pd.concat([low, high], axis=1)
connection = conection.iloc[:,[0,3, 8]]

In [123]:
connection.head()

,Community Board Name,Households With Low Speed Internet,Households With High Speed Internet
0,BX08,519,22930
1,BX12,309,25718
2,BX10,158,25397
3,BX11,449,24830
4,BX06,282,18654


In [124]:
# Harmonize the dataframes
cdcomp = cdcomp.replace(['13 QUEENS', '12 BRONX', '03 QUEENS', '12 MANHATTAN',
       '09 MANHATTAN', '08 QUEENS', '01 QUEENS', '07 BRONX', '01 BROOKLYN',
       '03 MANHATTAN', '09 QUEENS', '05 BRONX', '03 BROOKLYN',
       '07 MANHATTAN', '12 BROOKLYN', '09 BROOKLYN', '07 QUEENS',
       '15 BROOKLYN', '08 MANHATTAN', '10 BROOKLYN', '08 BROOKLYN',
       '04 MANHATTAN', '02 MANHATTAN', '12 QUEENS', '04 BRONX',
       '04 BROOKLYN', '01 BRONX', '10 QUEENS', '18 BROOKLYN',
       '11 BROOKLYN', '10 MANHATTAN', '11 QUEENS', '16 BROOKLYN',
       '05 BROOKLYN', '05 MANHATTAN', '08 BRONX', '09 BRONX',
       '17 BROOKLYN', '01 MANHATTAN', '05 QUEENS', '06 QUEENS', '02 BRONX',
       '11 BRONX', '04 QUEENS', '02 QUEENS', '01 STATEN ISLAND',
       '03 STATEN ISLAND', '14 BROOKLYN', '07 BROOKLYN',
       '02 STATEN ISLAND', '10 BRONX', '11 MANHATTAN', '06 BROOKLYN',
       '13 BROOKLYN', '06 MANHATTAN', '02 BROOKLYN', '06 BRONX',
       '14 QUEENS', '03 BRONX'], ['QN13', 'BX12', 'QN03', 'MN12', 'MN09', 'QN08', 'QN01', 'BX07', 'BK01',
        'MN03', 'QN09', 'BX05', 'BK03', 'MN07', 'BK12', 'BK09', 'QN07', 'BK15', 'MN08', 'BK10', 'BK08', 'MN04',
        'MN02', 'QN12', 'BX04', 'BK04', 'BX01', 'QN10', 'BK18', 'BK11', 'MN10', 'QN11', 'BK16', 'BK05', 'MN05',
        'BX08', 'BX09', 'BK17', 'MN01', 'QN05', 'QN06', 'BX02', 'BX11', 'QN04', 'QN02', 'SI01', 'SI03', 'BK14',
        'BK07', 'SI02', 'BX10', 'MN11', 'BK06', 'BK13', 'MN06', 'BK02', 'BX06', 'QN14', 'BX03'])

In [127]:
complaints = cdcomp.merge(demographics,on='Community Board Name',how='outer').merge(mobile,on='Community Board Name',how='outer').merge(connection,on='Community Board Name',how='outer')

In [128]:
complaints.head()

,Community Board Name,Count of Complaints,Median Household Income,Households With Mobile Broadband,Households Without Mobile Broadband,Households With Low Speed Internet,Households With High Speed Internet
0,MN12,1162,45157,26592.0,32179.0,1799.0,46246.0
1,QN12,968,52537,15241.0,39630.0,1817.0,40100.0
2,BK17,883,42349,13596.0,23007.0,622.0,30543.0
3,BK05,871,32996,9572.0,29604.0,512.0,32455.0
4,BK03,865,34678,12648.0,22363.0,754.0,25803.0


In [129]:
print(len(complaints))

71


### Are the demographics and infrastructure different in Community Districts that show more complaints than others?

Demographics and infrstructure are different in community districts that report more service requests to 311. We can see that the households located in the CDs with the largest amount of complaints, have a higher median income and better access to broadband. Conversely, households located within CDs with low complaint levels, have a lower median income and deficient access to high speed internet. 